In [41]:
import pandas as pd
import pyodbc
username = 'vigrose'
password = 'Hufflepuff10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)

In [5]:
iqvia_query = \
    """ 
    SELECT DISTINCT 
    B.BUSINESS_NAME,
    B.DBA_NAME,
    B.PHONE,
    B.PHYSICAL_ADDR_1,
    B.PHYSICAL_ADDR_2,
    B.PHYSICAL_CITY,
    B.PHYSICAL_STATE,
    B.PHYSICAL_ZIP,
    B.POSTAL_ADDR_1,
    B.POSTAL_ADDR_2,
    B.POSTAL_CITY,
    B.POSTAL_STATE,
    B.POSTAL_ZIP,
    P.ME,
    T.AFFIL_TYPE_DESC,
    A.AFFIL_IND,
    A.AFFIL_RANK
    FROM 
    ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
    WHERE  
    B.IMS_ORG_ID = A.IMS_ORG_ID
    AND
    A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
    AND
    A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
    AND
    P.CURRENT_BATCH_FLAG='Y'
    AND
    A.CURRENT_BATCH_FLAG='Y'
    AND
    B.CURRENT_BATCH_FLAG='Y'
    """

In [39]:
symphony_query = \
    """
    SELECT DISTINCT 
    L.OTHER_ID,
    D.ADDR_LINE_1_TXT,
    D.ADDR_LINE_2_TXT,
    D.ADDR_CITY_NAM,
    D.ADDR_ST_CDE,
    D.ADDR_ZIP_CDE,
    D.ADDR_FRST_TLPHN_NBR,
    D.ADDR_SCND_TLPHN_NBR
    FROM 
    ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT D, ODS.PRACTITIONER_ADDL_IDS_LAYOUT L
    WHERE  
    D.DS_PRCTR_ID=L.DS_PRCTR_ID
    AND
    D.ADDR_LINE_1_TXT IS NOT NULL
    AND
    L.ID_QLFR_TYP_CDE=38
    """

In [6]:
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,BUSINESS_NAME,DBA_NAME,PHONE,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,POSTAL_ADDR_1,POSTAL_ADDR_2,POSTAL_CITY,POSTAL_STATE,POSTAL_ZIP,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,SAINT VINCENTS SAINT CLAIRE,ALABAMA HEALTH SERVICES,2053383301,7063 VETERANS PKWY,None,PELL CITY,AL,35125-5114,7063 VETERANS PKWY,None,PELL CITY,AL,35125-5114,0010200001,Admitting,A,6
1,SOUTH BALDWIN REGIONAL MEDICAL CENTER,None,2519493400,1613 N MCKENZIE ST,None,FOLEY,AL,36535-2247,1613 N MCKENZIE ST,None,FOLEY,AL,36535-2247,0010200001,Admitting,A,6
2,PRATTVILLE BAPTIST HOSPITAL,None,3343650651,124 S MEMORIAL DR,None,PRATTVILLE,AL,36067-3619,124 S MEMORIAL DR,None,PRATTVILLE,AL,36067-3619,0010200001,Admitting,A,6
3,LARGO MEDICAL CENTER,None,7275885200,201 14TH ST SW,None,LARGO,FL,33770-3133,201 14TH ST SW,None,LARGO,FL,33770-3133,0010200001,Admitting,A,6
4,"MISSION HEALTH SYSTEM, INC",MISSION HOSPITAL SAINT JOSEPH CAMPUS,8282131111,428 BILTMORE AVE,None,ASHEVILLE,NC,28801-4502,428 BILTMORE AVE,None,ASHEVILLE,NC,28801-4502,0010200003,Admitting,A,6


In [42]:
symphony_2 = pd.read_sql(con=ODS, sql=symphony_query)
symphony_2.head()

,OTHER_ID,ADDR_LINE_1_TXT,ADDR_LINE_2_TXT,ADDR_CITY_NAM,ADDR_ST_CDE,ADDR_ZIP_CDE,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR
0,0010200001,4899 REYNOLDS LN,None,BIRMINGHAM,AL,35242,(727) 642-5087,None
1,0010200002,171 ASHLEY AVE,None,CHARLESTON,SC,29425,(843) 792-3537,None
2,0010200003,500 CENTRE PARK DR,None,ASHEVILLE,NC,28805,(828) 254-4337,None
3,0010200004,14502 W MEEKER BLVD,None,SUN CITY WEST,AZ,85375,None,None
4,0010200005,4429 CLARA ST,STE 440,NEW ORLEANS,LA,70115,(504) 842-9618,(504) 842-7444


In [12]:
XX = pd.merge(iqvia, symphony, left_on='ME',right_on='OTHER_ID')

In [13]:
XX = XX.fillna('None')

In [9]:
PPD = pd.read_csv("../../Data/PPD/ppd_data_20200718.csv")

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
PPD['ME']=fix_me(PPD.ME)

In [22]:
PPD = PPD.dropna(subset = ['POLO_MAILING_LINE_2'])
PPD = PPD.fillna('None')
tester = PPD[['ME','POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2','POLO_CITY', 'POLO_STATE', 'POLO_ZIP', 'TELEPHONE_NUMBER']].head(10)

In [55]:
tester = PPD[['ME?','POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2','POLO_CITY', 'POLO_STATE', 'POLO_ZIP', 'TELEPHONE_NUMBER']].head(10)

In [30]:
XX

,BUSINESS_NAME,DBA_NAME,PHONE,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK,OTHER_ID,ADDR_LINE_1_TXT,ADDR_LINE_2_TXT,ADDR_CITY_NAM,ADDR_ST_CDE,ADDR_ZIP_CDE,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR
0,SAINT VINCENTS SAINT CLAIRE,ALABAMA HEALTH SERVICES,2053383301,7063 VETERANS PKWY,None,PELL CITY,AL,35125-5114,0010200001,Admitting,A,6,0010200001,4899 REYNOLDS LN,None,BIRMINGHAM,AL,35242,(727) 642-5087,None
1,SOUTH BALDWIN REGIONAL MEDICAL CENTER,None,2519493400,1613 N MCKENZIE ST,None,FOLEY,AL,36535-2247,0010200001,Admitting,A,6,0010200001,4899 REYNOLDS LN,None,BIRMINGHAM,AL,35242,(727) 642-5087,None
2,PRATTVILLE BAPTIST HOSPITAL,None,3343650651,124 S MEMORIAL DR,None,PRATTVILLE,AL,36067-3619,0010200001,Admitting,A,6,0010200001,4899 REYNOLDS LN,None,BIRMINGHAM,AL,35242,(727) 642-5087,None
3,LARGO MEDICAL CENTER,None,7275885200,201 14TH ST SW,None,LARGO,FL,33770-3133,0010200001,Admitting,A,6,0010200001,4899 REYNOLDS LN,None,BIRMINGHAM,AL,35242,(727) 642-5087,None
4,"MISSION HEALTH SYSTEM, INC",MISSION HOSPITAL SAINT JOSEPH CAMPUS,8282131111,428 BILTMORE AVE,None,ASHEVILLE,NC,28801-4502,0010200003,Admitting,A,6,0010200003,500 CENTRE PARK DR,None,ASHEVILLE,NC,28805,(828) 254-4337,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452690,VCU HEALTH COMMUNITY MEMORIAL HOSPITAL,None,4344473151,1755 N MECKLENBURG AVE,None,SOUTH HILL,VA,23970-4080,9650196004,Attending,P,9,9650196004,142 S MAIN ST,None,DANVILLE,VA,24541,(434) 799-2225,None
1452691,WAKEMED HEALTH AND HOSPITALS,WAKEMED RALEIGH CAMPUS REHABILITATION HOSPITAL,9193508787,3000 NEW BERN AVE,None,RALEIGH,NC,27610-1231,9650196004,Attending,P,9,9650196004,142 S MAIN ST,None,DANVILLE,VA,24541,(434) 799-2225,None
1452692,THE BRENNER CHILDRENS HOSPITAL,None,3367136428,MEDICAL CENTER BLVD,None,WINSTON SALEM,NC,27157-0001,9650196005,IDN Affiliated (Inferred),A,8,9650196005,1 MEDICAL CENTER BLVD,None,WINSTON SALEM,NC,27103,(336) 716-2011,None
1452693,WAKE FOREST BAPTIST MEDICAL CENTER,None,3367162011,1 MEDICAL CENTER BLVD,None,WINSTON SALEM,NC,27157-0001,9650196005,Attending,P,9,9650196005,1 MEDICAL CENTER BLVD,None,WINSTON SALEM,NC,27103,(336) 716-2011,None


In [71]:
from fuzzywuzzy import fuzz

def count_found(row):
    '''count'''
    found = 0
    if row.PHYSICAL_STATE != 'None':
        found += 1
    if row.ADDR_ST_CDE != 'None':
        found += 1
    return found

def check_match(thing_1, thing_2):
    '''match check'''
    if thing_1 == thing_2:
        return 1
    elif thing_1 in thing_2 or thing_2 in thing_1:
        return 1
    elif fuzz.ratio(thing_1, thing_2) > 70:
        return 1
    else:
        return 0
    
def find_address_matches(new_addresses):
    '''Gahhh'''
    new_addresses = new_addresses.fillna('None')
    iqvia_zips = []
    symph_zips = []
    iqvia_adds = []
    symph_adds = []
    totals = []

    for row in new_addresses.itertuples():
        zip_symph = 0
        add_symph = 0
        zip_iq = 0
        add_iq = 0
    
        if row.POLO_STATE == row.ADDR_ST_CDE:
            if row.POLO_ZIP == row.ADDR_ZIP_CDE:
                zip_symph = 1
            if check_match(row.ADDR_LINE_1_TXT, row.POLO_MAILING_LINE_2):
                add_symph = 1
            if check_match(row.ADDR_LINE_1_TXT, row.POLO_MAILING_LINE_1):
                add_symph = 1

        if row.PHYSICAL_STATE == row.POLO_STATE:
            if row.PHYSICAL_ZIP == row.POLO_ZIP:
                zip_iq = 1
            if check_match(row.PHYSICAL_ADDR_1, row.POLO_MAILING_LINE_2):
                add_iq = 1
            if check_match(row.PHYSICAL_ADDR_2, row.POLO_MAILING_LINE_2):
                add_iq = 1

        iqvia_zips.append(zip_iq)
        symph_zips.append(zip_symph)
        iqvia_adds.append(add_iq)
        symph_adds.append(add_symph)
        totals.append(count_found(row))

    new_addresses['iqvia_Zip_Match'] = iqvia_zips
    new_addresses['iqvia_Address_2_Match'] = iqvia_adds
    new_addresses['symph_Zip_Match'] = symph_zips
    new_addresses['symph_Address_2_Match'] = symph_adds
    new_addresses['Total_Records'] = totals

    return new_addresses

In [69]:
test = pd.merge(XX, tester, left_on='OTHER_ID', right_on='ME?')

In [39]:
XX.ME

0          00010200001
1          00010200001
2          00010200001
3          00010200001
4          00010200003
              ...     
1452690    09650196004
1452691    09650196004
1452692    09650196005
1452693    09650196005
1452694    09650198002
Name: ME, Length: 1452695, dtype: object

In [51]:
PPD['ME?']

1          0010200002
2          0010200003
3          0010200004
4          0010200005
5          0010200006
              ...    
1283760    9650196001
1283762    9650196003
1283763    9650196004
1283764    9650196005
1283765    9650198002
Name: ME?, Length: 981212, dtype: object

In [37]:
XX['ME']=fix_me(XX.ME)

In [40]:
XX.OTHER_ID

0          0010200001
1          0010200001
2          0010200001
3          0010200001
4          0010200003
              ...    
1452690    9650196004
1452691    9650196004
1452692    9650196005
1452693    9650196005
1452694    9650198002
Name: OTHER_ID, Length: 1452695, dtype: object

In [45]:
PPD[PPD['ME']=='9650198002']

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST


In [47]:
PPD['ME?']=[x[0:10] for x in PPD.ME]

In [48]:
PPD['ME?']

1          0010200002
2          0010200003
3          0010200004
4          0010200005
5          0010200006
              ...    
1283760    9650196001
1283762    9650196003
1283763    9650196004
1283764    9650196005
1283765    9650198002
Name: ME?, Length: 981212, dtype: object

In [72]:
find_address_matches(test)

,BUSINESS_NAME,DBA_NAME,PHONE,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,ME,AFFIL_TYPE_DESC,...,iqvia_State_Match,iqvia_City_Match,iqvia_Zip_Match,iqvia_Address_1_Match,iqvia_Address_2_Match,symph_State_Match,symph_City_Match,symph_Zip_Match,symph_Address_2_Match,Total_Records
0,"MISSION HEALTH SYSTEM, INC",MISSION HOSPITAL SAINT JOSEPH CAMPUS,8282131111,428 BILTMORE AVE,None,ASHEVILLE,NC,28801-4502,00010200003,Admitting,...,1,1,0,0,0,1,1,1,1,2
1,MISSION HEALTH,MISSION HOSPITAL,8282131111,509 BILTMORE AVE,None,ASHEVILLE,NC,28801-4601,00010200003,Admitting,...,1,1,0,0,0,1,1,1,1,2
2,BANNER DEL E WEBB MEDICAL CENTER,None,6235244000,14502 W MEEKER BLVD,None,SUN CITY WEST,AZ,85375-5282,00010200004,Attending,...,1,1,0,0,1,1,1,1,1,2
3,OCHSNER EXTENDED CARE HOSPITAL,None,5044648590,2614 JEFFERSON HWY,None,JEFFERSON,LA,70121-3828,00010200005,IDN Affiliated (Inferred),...,1,0,0,0,0,1,1,1,1,2
4,OCHSNER HEALTH SYSTEM,None,5048423000,1516 JEFFERSON HWY,None,NEW ORLEANS,LA,70121-2429,00010200005,IDN Affiliated (Inferred),...,1,1,0,0,0,1,1,1,1,2
5,OCHSNER BAPTIST MEDICAL CENTER,None,5048999311,2700 NAPOLEON AVE,None,NEW ORLEANS,LA,70115-6914,00010200005,IDN Affiliated,...,1,1,0,1,0,1,1,1,1,2
6,SINGING RIVER HOSPITAL,None,2288095000,2809 DENNY AVE,None,PASCAGOULA,MS,39581-5301,00010200008,Admitting,...,0,0,0,0,0,1,1,1,1,2
7,OCEAN SPRINGS HOSPITAL,SINGING RIVER HEALTH SYSTEM,2288181111,3109 BIENVILLE BLVD,None,OCEAN SPRINGS,MS,39564-4361,00010200008,Admitting,...,0,0,0,0,0,1,1,1,1,2
8,MONROE CARELL JR CHILDRENS HOSPITAL AT VANDERBILT,None,6159361000,2200 CHILDRENS WAY,None,NASHVILLE,TN,37232-0005,00010200009,Attending,...,1,1,0,0,1,1,1,1,1,2
9,MARSHALL MEDICAL CENTER NORTH,None,2565718000,8000 AL HIGHWAY 69,None,GUNTERSVILLE,AL,35976-7140,00010200011,Attending,...,1,1,0,0,1,1,1,1,1,2


In [62]:
test.columns

Index(['BUSINESS_NAME', 'DBA_NAME', 'PHONE', 'PHYSICAL_ADDR_1',
       'PHYSICAL_ADDR_2', 'PHYSICAL_CITY', 'PHYSICAL_STATE', 'PHYSICAL_ZIP',
       'ME', 'AFFIL_TYPE_DESC', 'AFFIL_IND', 'AFFIL_RANK', 'OTHER_ID',
       'ADDR_LINE_1_TXT', 'ADDR_LINE_2_TXT', 'ADDR_CITY_NAM', 'ADDR_ST_CDE',
       'ADDR_ZIP_CDE', 'ADDR_FRST_TLPHN_NBR', 'ADDR_SCND_TLPHN_NBR', 'ME?',
       'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE',
       'POLO_ZIP', 'TELEPHONE_NUMBER'],
      dtype='object')

In [5]:
iqvia.drop_duplicates(['BUSINESS_NAME', 'PHYSICAL_ADDR_1']).to_csv('../../Data/MasterfileCore/Aah.csv')

In [8]:
len(symphony)

33330151

In [11]:
pd.merge(symphony, PPD, left_on='OTHER_ID', right_on='ME')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [12]:
PPD.dtypes

ME                                  int64
RECORD_ID                          object
UPDATE_TYPE                         int64
ADDRESS_TYPE                      float64
MAILING_NAME                       object
                                   ...   
MOST_RECENT_FORMER_MIDDLE_NAME     object
MOST_RECENT_FORMER_FIRST_NAME      object
NEXT_MOST_RECENT_FORMER_LAST       object
NEXT_MOST_RECENT_FORMER_MIDDLE     object
NEXT_MOST_RECENT_FORMER_FIRST      object
Length: 81, dtype: object

In [15]:
symphony['ME']=fix_me(symphony['OTHER_ID'])

In [ ]:
symphony[0:20]

In [18]:
PPD.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [23]:
symphony['OTHER_ID']

0                  #1267
1                  #2003
2                  #2009
3             (S)D.O.000
4           (S)D.O.00004
                ...     
33330146          ZZ2737
33330147          ZZ2741
33330148          ZZ4221
33330149          ZZ8267
33330150          ZZA993
Name: OTHER_ID, Length: 33330151, dtype: object

In [24]:
PPD.ME

0          00102000013
1          00102000021
2          00102000030
3          00102000048
4          00102000056
              ...     
1283761    96501960028
1283762    96501960036
1283763    96501960044
1283764    96501960052
1283765    96501980029
Name: ME, Length: 1283766, dtype: object

In [25]:
SYMPH = symphony[symphony['ME'].isin(PPD.ME)]

In [ ]:
def find_address_matches(new_addresses):
    '''Gahhh'''Zipcode   
    new_addresses = new_addresses.fillna('None')
    amaia_zips = []
    dhc_zips = []
    amaia_adds = []
    dhc_adds = []
    iqvia_zips = []
    symph_zips = []
    iqvia_adds = []
    symph_adds = []
    totals = []

    for row in new_addresses.itertuples():
        zip_dhc = 0
        add_dhc = 0
        zip_ins = 0
        add_ins = 0
        zip_symph = 0
        add_symph = 0
        zip_iq = 0
        add_iq = 0

        if row.State_New == row.State_DHC:
            if row.Zipcode == row.Zip_Code:
                zip_dhc = 1
            if check_match(row.Address, row.Address_2):
                add_dhc = 1
        if row.STATE_CD == row.State_New:
            if row.ZIP == row.Zipcode:
                zip_ins = 1
            if check_match(row.ADDR_1, row.Address_2):
                add_ins = 1
            if check_match(row.ADDR_2, row.Address_2):
                add_ins = 1
        if row.POLO_STATE == row.ADDR_ST_CDE:
            if row.POLO_ZIP == row.ADDR_ZIP_CDE:
                zip_symph = 1
            if check_match(row.ADDR_LINE_1_TXT, row.POLO_MAILING_LINE_2):
                add_symph = 1
            if check_match(row.ADDR_LINE_1_TXT, row.POLO_MAILING_LINE_1):
                add_symph = 1
        if row.PHYSICAL_STATE == row.POLO_STATE:
            if row.PHYSICAL_ZIP == row.POLO_ZIP:
                zip_iq = 1
            if check_match(row.PHYSICAL_ADDR_1, row.POLO_MAILING_LINE_2):
                add_iq = 1
            if check_match(row.PHYSICAL_ADDR_2, row.POLO_MAILING_LINE_2):
                add_iq = 1

        amaia_zips.append(zip_ins)
        dhc_zips.append(zip_dhc)
        amaia_adds.append(add_ins)
        dhc_adds.append(add_dhc)
        iqvia_zips.append(zip_iq)
        symph_zips.append(zip_symph)
        iqvia_adds.append(add_iq)
        symph_adds.append(add_symph)
        
        totals.append(count_found(row))

    new_addresses['AMAIA_Zip_Match'] = amaia_zips
    new_addresses['AMAIA_Address_2_Match'] = amaia_adds
    new_addresses['DHC_Zip_Match'] = dhc_zips
    new_addresses['DHC_Address_2_Match'] = dhc_adds
    new_addresses['iqvia_Zip_Match'] = iqvia_zips
    new_addresses['iqvia_Address_2_Match'] = iqvia_adds
    new_addresses['symph_Zip_Match'] = symph_zips
    new_addresses['symph_Address_2_Match'] = symph_adds
    new_addresses['Total_Records'] = totals

    return new_addresses

In [26]:
SYMPH

,OTHER_ID,ADDR_LINE_1_TXT,ADDR_LINE_2_TXT,ADDR_CITY_NAM,ADDR_ST_CDE,ADDR_ZIP_CDE,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR,ME
917962,0102050011,14800 PHYSICIANS LN,STE 231,ROCKVILLE,MD,20850,(301) 762-6686,None,00102050011
917984,0102050037,4518 UNION DEPOSIT RD,None,HARRISBURG,PA,17111,(717) 652-5840,None,00102050037
917991,0102050045,100 HITCHCOCK WAY,None,MANCHESTER,NH,03104,(603) 695-2500,(603) 695-2650,00102050045
917999,0102050053,4058 BAY HILL DR,None,ABILENE,TX,79606,(540) 539-4030,None,00102050053
918006,0102050061,7710 MERCY RD,STE 201,OMAHA,NE,68124,(402) 391-5022,None,00102050061
...,...,...,...,...,...,...,...,...,...
18615155,6801070017,30741 TANGLEWOOD TRL,None,FARMINGTON,MI,48331,None,None,06801070017
18615161,6801070025,1350 E WEST MAPLE RD,STE 5,WALLED LAKE,MI,48390,(248) 496-7558,None,06801070025
18620725,6801080012,1308 PEARL ST,None,YPSILANTI,MI,48197,None,None,06801080012
18620730,6801080021,413 E FLORENCE BLVD,None,CASA GRANDE,AZ,85122,(520) 876-5764,None,06801080021


In [27]:
pd.merge(SYMPH, PPD, on='ME')

,OTHER_ID,ADDR_LINE_1_TXT,ADDR_LINE_2_TXT,ADDR_CITY_NAM,ADDR_ST_CDE,ADDR_ZIP_CDE,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR,ME,RECORD_ID,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
0,0102050011,14800 PHYSICIANS LN,STE 231,ROCKVILLE,MD,20850,(301) 762-6686,None,00102050011,A,...,VT,05401,5501.0,C007,CHAMBLESS,LEA,HEATHER,NaN,NaN,NaN
1,0102050037,4518 UNION DEPOSIT RD,None,HARRISBURG,PA,17111,(717) 652-5840,None,00102050037,A,...,GA,30106,1114.0,C001,NaN,NaN,NaN,NaN,NaN,NaN
2,0102050045,100 HITCHCOCK WAY,None,MANCHESTER,NH,03104,(603) 695-2500,(603) 695-2650,00102050045,A,...,AL,35801,6429.0,C011,NaN,NaN,NaN,NaN,NaN,NaN
3,0102050053,4058 BAY HILL DR,None,ABILENE,TX,79606,(540) 539-4030,None,00102050053,A,...,MS,39216,4500.0,C099,STRONACH,NaN,BEN,NaN,NaN,NaN
4,0102050061,7710 MERCY RD,STE 201,OMAHA,NE,68124,(402) 391-5022,None,00102050061,A,...,AL,36109,2707.0,C061,BAILEY,TAMARA,JESSAKA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4461,6801070017,30741 TANGLEWOOD TRL,None,FARMINGTON,MI,48331,None,None,06801070017,A,...,MO,65804,2203.0,C075,NaN,NaN,NaN,NaN,NaN,NaN
4462,6801070025,1350 E WEST MAPLE RD,STE 5,WALLED LAKE,MI,48390,(248) 496-7558,None,06801070025,A,...,OR,97239,4501.0,C066,NaN,NaN,NaN,NaN,NaN,NaN
4463,6801080012,1308 PEARL ST,None,YPSILANTI,MI,48197,None,None,06801080012,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4464,6801080021,413 E FLORENCE BLVD,None,CASA GRANDE,AZ,85122,(520) 876-5764,None,06801080021,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
pd.merge(symphony, party_key, on='OTHER_ID')

,OTHER_ID,ADDR_LINE_1_TXT,ADDR_LINE_2_TXT,ADDR_CITY_NAM,ADDR_ST_CDE,ADDR_ZIP_CDE,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR,ME_x,PARTY_ID,ME_y
0,11100654,12605 E 16TH AVE,None,AURORA,CO,80045,(720) 848-0000,None,00011100654,11100654,3.005201e+09
1,1800190,7336 HOLZHAUER RD,None,NORTHFIELD,OH,44067,None,None,1800190,1800190,1.029904e+08
2,1800192,305 RICHLAND ST,None,STEUBENVILLE,OH,43952,None,None,1800192,1800192,1.029906e+08
3,1800356,145 S PINECREST ST,None,WICHITA,KS,67218,None,None,1800356,1800356,1.060106e+08
4,1800532,8908 W 124TH ST,APT 88,OVERLAND PARK,KS,66213,None,None,1800532,1800532,3.010001e+08
...,...,...,...,...,...,...,...,...,...,...,...
95514,9901036,9003 WESTON PKWY,None,CARY,NC,27513,(919) 677-1459,(919) 677-1400,9901036,9901036,3.875141e+09
95515,9901037,1364 WHISPERING PINES LN,STE 3,GRASS VALLEY,CA,95945,(530) 272-3111,None,9901037,9901037,1.176150e+09
95516,9901038,508 FULTON ST,None,DURHAM,NC,27705,None,None,9901038,9901038,1.275151e+09
95517,9901039,767 W 1ST ST,None,NEWTON,NC,28658,(828) 465-3928,None,9901039,9901039,1.275140e+09


In [29]:
party_key = pd.read_csv('../../Data/Triangulation/Party_ME.csv')

In [35]:
party_key['OTHER_ID']=[str(x) for x in party_key.PARTY_ID]

In [32]:
symphony.dtypes

OTHER_ID               object
ADDR_LINE_1_TXT        object
ADDR_LINE_2_TXT        object
ADDR_CITY_NAM          object
ADDR_ST_CDE            object
ADDR_ZIP_CDE           object
ADDR_FRST_TLPHN_NBR    object
ADDR_SCND_TLPHN_NBR    object
ME                     object
dtype: object

In [38]:
symphony['OTHER_ID']

0                  #1267
1                  #2003
2                  #2009
3             (S)D.O.000
4           (S)D.O.00004
                ...     
33330146          ZZ2737
33330147          ZZ2741
33330148          ZZ4221
33330149          ZZ8267
33330150          ZZA993
Name: OTHER_ID, Length: 33330151, dtype: object

In [43]:
symphony_2

,OTHER_ID,ADDR_LINE_1_TXT,ADDR_LINE_2_TXT,ADDR_CITY_NAM,ADDR_ST_CDE,ADDR_ZIP_CDE,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR
0,0010200001,4899 REYNOLDS LN,None,BIRMINGHAM,AL,35242,(727) 642-5087,None
1,0010200002,171 ASHLEY AVE,None,CHARLESTON,SC,29425,(843) 792-3537,None
2,0010200003,500 CENTRE PARK DR,None,ASHEVILLE,NC,28805,(828) 254-4337,None
3,0010200004,14502 W MEEKER BLVD,None,SUN CITY WEST,AZ,85375,None,None
4,0010200005,4429 CLARA ST,STE 440,NEW ORLEANS,LA,70115,(504) 842-9618,(504) 842-7444
...,...,...,...,...,...,...,...,...
1294352,9650196002,865 S 1ST ST,None,JESUP,GA,31545,(912) 427-6811,None
1294353,9650196003,730 W MARKET ST,None,LIMA,OH,45801,(419) 226-4310,None
1294354,9650196004,142 S MAIN ST,None,DANVILLE,VA,24541,(434) 799-2225,None
1294355,9650196005,1 MEDICAL CENTER BLVD,None,WINSTON SALEM,NC,27103,(336) 716-2011,None


In [46]:
PPD['OTHER_ID']=[x[0:10] for x in PPD.ME]

In [47]:
PPD.OTHER_ID

0          0010200001
1          0010200002
2          0010200003
3          0010200004
4          0010200005
              ...    
1283761    9650196002
1283762    9650196003
1283763    9650196004
1283764    9650196005
1283765    9650198002
Name: OTHER_ID, Length: 1283766, dtype: object

In [52]:
ppd = PPD[['ME','OTHER_ID']]

In [53]:
SYMPH = pd.merge(symphony_2, ppd, on='OTHER_ID')

In [51]:
iqvia = iqvia[iqvia.ME!='None']

In [57]:
IQVIA = pd.merge(iqvia, ppd, left_on='ME', right_on='OTHER_ID', suffixes = ('_iqvia',''))

In [58]:
SYMPH.to_csv("../../Data/Triangulation/Symphony_200828.csv",index=False)
IQVIA.to_csv("../../Data/Triangulation/IQVia_200828.csv",index=False)

In [60]:
sample = pd.read_csv('../../Data/Triangulation/sample_data.csv')

In [61]:
sample.columns

Index(['sample_id', 'row_id', 'survey_month', 'survey_year', 'survey_type',
       'sample_source', 'me', 'entity_id', 'first_name', 'middle_name',
       'last_name', 'suffix', 'polo_comm_id', 'Address_1', 'Address_2', 'City',
       'State', 'Zipcode', 'phone_comm_id', 'telephone_number', 'prim_spec_cd',
       'description', 'pe_cd', 'fax_number'],
      dtype='object')

In [72]:
sample = sample.fillna('None')
new_list =[]
for row in sample.drop_duplicates(subset = 'polo_comm_id').itertuples():
    address_key = row.Address_2 + '_' + row.Zipcode
    polo_comm_id = row.polo_comm_id
    new_dict = {
        'ADDRESS_KEY': address_key,
        'polo_comm_id': polo_comm_id
    }
    new_list.append(new_dict)

In [67]:
len(sample.drop_duplicates(subset = 'polo_comm_id'))

16845

In [73]:
new_df = pd.DataFrame(new_list)
pd.merge(sample, new_df, on='polo_comm_id').to_csv('../../Data/Triangulation/sample_2.csv', index=False)